# A Pythonic Object

## Vector Classs

In [1]:
from array import array
import math

In [2]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}{!r}, {!r}'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

> typecode是类属性，在Vector2d实例和字节序列之间转换时使用

## An Alternative Constructor

```python
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)
```

## classmethod and staticmethod

classmethod定义操作类，而不是操作实例的方法。classmethod改变了调用方法的方式，因此类方法的第一个参数时类本事，而不是实例。classmethod最常见的用途是定义备选构造方法。

staticmethod装饰器也会改变调用方式，但是第一个参数不是特殊值。静态方法就是普通的函数，只是放在类的定义体中。

## Formatted Display

[PEP Advanced String Formatting](https://www.python.org/dev/peps/pep-3101/)

格式规范语言为一些内置类型提供了专用的表示代码。比如，b和x分别表示二进制和十六进制的int类型，f表示小数形式的float类型，而%表示百分数形式。

In [9]:
help(format)

Help on built-in function format in module builtins:

format(value, format_spec='', /)
    Return value.__format__(format_spec)
    
    format_spec defaults to the empty string.
    See the Format Specification Mini-Language section of help('FORMATTING') for
    details.



In [7]:
format(42, 'b')

'101010'

In [12]:
format(2/3, '.2%')

'66.67%'

格式规范语言是扩展的，因为各个类可以自行决定如何解释format_spec参数。例如：datetime模块中类，它们的`__format__`方法使用的格式嗲吗与strftime()函数一样。

In [32]:
import datetime

now = datetime.datetime.now()
format(now, '%H%:%M:%S')

'16:45:13'

In [35]:
"It's now {:%I:%M %p}".format(now)

"It's now 04:45 PM"

如果类没有定义`__format__`方法，从object继承的方法会返回str(my_object)。我们为Vector2d定义了`__str__`方法：

In [36]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

然而，如果传入格式说明符，`object.__format__`方法会抛出TypeError:

In [38]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

```python
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components)
```

## A Hashed Vector2d

注意，我们让这些向量不可变是有原因的，因为这样才能实现`__hash__`方法。这个方法应该返回一个整数，理想情况下还要考虑对象属性的散列值(`__eq__`方法也要使用)，因为相等的对象应该具有相同的散列值。根据特殊方法`__hash__`的文档，最好使用位运算符异或（^）混合各分量的散列值。

```python
def __hash__(self):
    return hash(self.x) ^ hash(self.y)
```

> 要想创建可散列的类型，不一定要实现特性，也不一定保护实例属性。只需正确的实现`__hash__`和`__eq__`方法即可。但是，实例的散列值绝不应该变化。

## Reference

* [Advanced String Formatting](https://www.python.org/dev/peps/pep-3101/)